## Install dependencies

In [ ]:
!pip install openai
!pip install langchain
!pip install unstructured
!pip install tiktoken
!pip install chromadb

## Imports

In [1]:
from pathlib import Path

from IPython.display import display, Markdown

from langchain.vectorstores import Chroma
from langchain.text_splitter import MarkdownTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader, UnstructuredHTMLLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document

import openai
import tiktoken

# Store OpenAI key in env var: OPENAI_API_KEY

## Load Docs

#### First, we load the docs and split any that are longer than 1500 characters
This could also be done with a different format of doc, by using a different splitter

Note: Because ChatGPT can understand markdown, we are loading the docs with the raw
`TextLoader` instead of something like the `UnstructuredMarkdownLoader` which strips
out the markdown and leaves plain text.

In [2]:
# Load the docs
loader = DirectoryLoader('html_pages/', glob="**/*", loader_cls=UnstructuredHTMLLoader)
docs = loader.load()

In [3]:
# Split the docs into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=250)
docs = text_splitter.split_documents(docs)

#### Then, we create a ChromaDB vector store with source metadata

In [16]:
# This will create the Chroma vector database with embeddings for each chunk of text

embeddings = OpenAIEmbeddings() # leave embeddings on default text-embedding-ada-002

database_persistent_directory = ".html_db_storage"
vector_db = Chroma.from_texts([doc.page_content for doc in docs], embeddings, persist_directory=database_persistent_directory, metadatas=[{"source": f"https://www.multiamory.com/podcast/{Path(*Path(doc.metadata['source']).parts[1:]).as_posix()}" } for doc in docs])

vector_db.persist()

Using embedded DuckDB with persistence: data will be stored in: .ma_db_storage


In [28]:
# If changes have been persisted already, we can load from local storage instead of re-creating the database
database_persistent_directory = ".ma_db_storage"

vector_db = Chroma(persist_directory=database_persistent_directory, embedding_function=embeddings)

Using embedded DuckDB with persistence: data will be stored in: .db_storage


## OpenAI Functions

Some custom functions to make interacting with the OpenAI API easier

In [8]:
from typing import List, Dict

# This function is mostly just so we can use the "stream" option to get responses in real-time.

def get_response(question: str, history: List[Dict[str, str]], model: str="gpt-3.5-turbo", temperature: float=0.5, stream: bool=True, timeout=None):
    if not timeout:
        timeout = 2.0 if stream else 60.0
    response = openai.ChatCompletion.create(
        model= model,
        messages= history + [
            {'role': 'user', 'content': f"{question}"},
        ],
        temperature=temperature,
        stream=stream,
        request_timeout=timeout
    )

    LINE_LENGTH = 80
    this_line_length = 0
    full_response = ""
    if not stream:
        response = [{"choices": [{"delta": {"content": f"{chunk} "}}]} for chunk in response["choices"][0]["message"]["content"].split(" ")]
    for chunk in response:
        chunk_text = chunk["choices"][0]["delta"].get("content", "")
        full_response += chunk_text

        if "\n" in chunk_text:
            parts = chunk_text.split("\n")
            for part in parts[:-1]:
                print(f"{part}\n", end='', flush=True)
            this_line_length = 0
            chunk_text = parts[-1]
        if this_line_length + len(chunk_text) > LINE_LENGTH:
            first_char = chunk_text[:1] if chunk_text[:1] in (" ", ".", "!", "?", ",", ";", ":") else ""
            print(f"{first_char}", end='\n', flush=True)
            chunk_text = chunk_text[len(first_char):]
            this_line_length = 0
        this_line_length += len(chunk_text)
        print(f"{chunk_text}", end='', flush=True)
    return full_response

In [9]:
def num_tokens_in_text(text):
    encoding = tiktoken.get_encoding("cl100k_base") # This is encoding for the chat models
    return len(encoding.encode(text))

## Document Lookup Examples

In [ ]:
# This returns the top k most similar documents to the query

doc_lookup = vector_db.similarity_search(query="What are the best ways to improve communication in a long-term relationship?", k=4)
for doc in doc_lookup:
    print(f"{doc.metadata['source']}")
    print(f"{doc.page_content}")
    print("")

In [ ]:
# the MMR function returns the top k most similar documents to the query, but with a diversity penalty to avoid returning documents that are too similar to each other

doc_lookup_mmr = vector_db.max_marginal_relevance_search(query="What are the best ways to improve communication in a long-term relationship?", k=4)
for doc in doc_lookup:
    print(f"{doc.metadata['source']}")
    print(f"{doc.page_content}")
    print("")

## TEMPLATES

In [31]:
def create_document_list(documents: List[Document], max_tokens = 1000):
    template = "----------------------------------------\nDocument {index}: {source}\n{doc.page_content}\n\n"
    total_tokens = 0
    result = ""
    for index, doc in enumerate(documents):
        doc_text = template.format(index=index+1, source=doc.metadata["source"].replace("multiamory_episode_pages/", ""), doc=doc)
        total_tokens += num_tokens_in_text(doc_text)
        if total_tokens > max_tokens:
            print(f"Truncating document list to fit within max tokens. Doc list limited to {index+1} documents.")
            break
        result += doc_text
    return result

## Question Asking

In [32]:
def question_from_sources(question):
    doc_lookup = vector_db.max_marginal_relevance_search(query=question, k=4)
    documents = create_document_list(doc_lookup, max_tokens=1000)
    prompt = f"Here are some relevant excerpts from the Multiamory podcast. Based on those as well as your existing knowledge, answer this question: {question}\n\n" + documents
    print("Generating answer using excerpts from the following episodes:\n" + "\n".join([f"{doc.metadata['source']}" for doc in doc_lookup]))
    response = get_response(prompt, [{"role": "system", "content": "You are a compassionate and helpful relationship coach who references the Multiamory Podcast and is knowledgable about relationships. You try to give practical answers and advice to listener questions."}], model="gpt-3.5-turbo", temperature=0.5, stream=True)
    return response

In [33]:
res = question_from_sources("Where should we go to eat today?")

Truncating document list to fit within max tokens. Doc list limited to 3 documents.
Generating answer using excerpts from the following episodes:
https://www.multiamory.com/podcast/multiamory_episode_pages/255-why-you-make-bad-decisions
https://www.multiamory.com/podcast/multiamory_episode_pages/255-why-you-make-bad-decisions
https://www.multiamory.com/podcast/multiamory_episode_pages/210-take-the-fight-out-of-your-fights
https://www.multiamory.com/podcast/379-relationship-science-for-your-friendships-and-vice-versa
Based on the excerpts from the Multiamory podcast, a helpful suggestion for 
where to go to eat today would be to use the collaborative decision-making 
process described by Dedeker, Jase, and Emily. One person could come up with 
five options, and the other person could pick two of those options. Then, the 
original person could pick one from those two. This process simplifies decision
-making and ensures that both people have a say in the final decision. 
Alternatively, i